In [1]:
import sys
# use line-buffering for both stdout and stderr
# sys.stdout = open(sys.stdout.fileno(), mode='w', buffering=1)
# sys.stderr = open(sys.stderr.fileno(), mode='w', buffering=1)

import hydra
from omegaconf import OmegaConf
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
import pathlib
from diffusion_policy.workspace.base_workspace import BaseWorkspace
 
import os
import hydra
import torch
from omegaconf import OmegaConf
import pathlib
from torch.utils.data import DataLoader
import copy
import random
import wandb
import tqdm
import numpy as np
import shutil
# from diffusion_policy.workspace.base_workspace import BaseWorkspace
from diffusion_policy.policy.robomimic_lowdim_policy import RobomimicLowdimPolicy
from diffusion_policy.dataset.base_dataset import BaseLowdimDataset
from diffusion_policy.env_runner.base_lowdim_runner import BaseLowdimRunner
from diffusion_policy.common.checkpoint_util import TopKCheckpointManager
from diffusion_policy.common.json_logger import JsonLogger
from diffusion_policy.common.pytorch_util import dict_apply, optimizer_to
import json

/home/ns1254/miniconda3/envs/robodiff/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /home/ns1254/diffusion_policy/envs/robomimic/robomimic/scripts/setup_macros.py
)


In [2]:
OmegaConf.register_new_resolver("eval", eval, replace=True)

In [3]:
# python train.py --config-dir="configs" --config-name=config_lowdim_can_mh_bc.yaml training.seed=42 training.device=cuda:0 hydra.run.dir='data/outputs/${now:%Y.%m.%d}/${now:%H.%M.%S}_${name}_${task_name}' +mask_fn=/home/carl_lab/ns/diffusion_policy/data/can_mh_bed_masked_0_60.txt +segments_toremove_file="/home/ns1254/gib/gib_results/gib_can_mh_image/subtask_rm120/segs_index_rm120_gib_can_mh_image.json"

config_file = "config_lowdim_can_mh_bc.yaml"

config_path='configs'

OmegaConf.register_new_resolver("eval", eval, replace=True)

with initialize(version_base=None, config_path=config_path):
    cfg = compose(config_name=config_file, overrides=[])
    print(cfg)

OmegaConf.resolve(cfg)

{'_target_': 'diffusion_policy.workspace.train_robomimic_lowdim_workspace.TrainRobomimicLowdimWorkspace', 'action_dim': 7, 'checkpoint': {'save_last_ckpt': True, 'save_last_snapshot': False, 'topk': {'format_str': 'epoch={epoch:04d}-test_mean_score={test_mean_score:.3f}.ckpt', 'k': 5, 'mode': 'max', 'monitor_key': 'test_mean_score'}}, 'dataloader': {'batch_size': 256, 'num_workers': 1, 'persistent_workers': False, 'pin_memory': True, 'shuffle': True}, 'exp_name': 'default', 'horizon': 1, 'keypoint_visible_rate': 1.0, 'logging': {'group': None, 'id': None, 'mode': 'online', 'name': '2022.12.30-01.51.31_train_robomimic_lowdim_can_lowdim', 'project': 'diffusion_policy_debug', 'resume': True, 'tags': ['train_robomimic_lowdim', 'can_lowdim', 'default']}, 'multi_run': {'run_dir': 'data/outputs/2022.12.30/01.51.31_train_robomimic_lowdim_can_lowdim', 'wandb_name_base': '2022.12.30-01.51.31_train_robomimic_lowdim_can_lowdim'}, 'n_action_steps': 1, 'n_latency_steps': 0, 'n_obs_steps': 1, 'name':

In [4]:
segs_fn="/home/ns1254/gib/gib_results/gib_can_mh_image/subtask_rm120/segs_index_rm120_gib_can_mh_image.json"
segments_toremove_file = segs_fn
if hasattr(cfg, 'segments_toremove_file'):
    segments_toremove_file = cfg.segments_toremove_file
    if segments_toremove_file is None:
        segments_toremove_file=""
    
if os.path.exists(segments_toremove_file):
    print('using segs file: ', segments_toremove_file)
    with open(segments_toremove_file, 'r') as f:
        data = json.load(f) 
    segs_toremove = data['data'] 
else:
    segs_toremove = {}  

print(f"segs_toremove: {segs_toremove}") 

using segs file:  /home/ns1254/gib/gib_results/gib_can_mh_image/subtask_rm120/segs_index_rm120_gib_can_mh_image.json
segs_toremove: {'demo_228': [[0, 119], [119, 385]], 'demo_243': [[0, 160], [160, 302]], 'demo_202': [[0, 299], [299, 397]], 'demo_122': [[0, 83]], 'demo_201': [[0, 125], [125, 531]], 'demo_71': [[0, 111], [111, 605]], 'demo_203': [[0, 131], [131, 261]], 'demo_208': [[0, 290], [290, 706]], 'demo_226': [[0, 196], [196, 286]], 'demo_116': [[0, 70]], 'demo_151': [[0, 80]], 'demo_248': [[248, 597]], 'demo_216': [[0, 118], [118, 206]], 'demo_209': [[0, 197], [197, 335]], 'demo_220': [[0, 150], [150, 270]], 'demo_234': [[0, 160], [160, 257]], 'demo_64': [[129, 1050]], 'demo_69': [[0, 286], [286, 914]], 'demo_87': [[0, 174], [174, 328]], 'demo_80': [[0, 206], [206, 335]], 'demo_219': [[0, 186]], 'demo_229': [[71, 488]], 'demo_273': [[140, 398]], 'demo_70': [[0, 177], [177, 303]], 'demo_236': [[0, 136], [136, 225]], 'demo_244': [[0, 351], [351, 445]], 'demo_65': [[117, 243]], 'de

In [5]:
cls = hydra.utils.get_class(cfg._target_)
cls

diffusion_policy.workspace.train_robomimic_lowdim_workspace.TrainRobomimicLowdimWorkspace

In [6]:
new_cfg={key:value for key, value in cfg.items()}
del new_cfg['_target_']
new_cfg

{'action_dim': 7,
 'checkpoint': {'save_last_ckpt': True, 'save_last_snapshot': False, 'topk': {'format_str': 'epoch={epoch:04d}-test_mean_score={test_mean_score:.3f}.ckpt', 'k': 5, 'mode': 'max', 'monitor_key': 'test_mean_score'}},
 'dataloader': {'batch_size': 256, 'num_workers': 1, 'persistent_workers': False, 'pin_memory': True, 'shuffle': True},
 'exp_name': 'default',
 'horizon': 1,
 'keypoint_visible_rate': 1.0,
 'logging': {'group': None, 'id': None, 'mode': 'online', 'name': '2022.12.30-01.51.31_train_robomimic_lowdim_can_lowdim', 'project': 'diffusion_policy_debug', 'resume': True, 'tags': ['train_robomimic_lowdim', 'can_lowdim', 'default']},
 'multi_run': {'run_dir': 'data/outputs/2022.12.30/01.51.31_train_robomimic_lowdim_can_lowdim', 'wandb_name_base': '2022.12.30-01.51.31_train_robomimic_lowdim_can_lowdim'},
 'n_action_steps': 1,
 'n_latency_steps': 0,
 'n_obs_steps': 1,
 'name': 'train_robomimic_lowdim',
 'obs_dim': 23,
 'past_action_visible': False,
 'policy': {'_target

In [7]:
# from diffusion_policy.workspace.train_robomimic_lowdim_workspace import TrainRobomimicLowdimWorkspace

In [8]:
import os
import hydra
import torch
from omegaconf import OmegaConf
import pathlib
from torch.utils.data import DataLoader
import copy
import random
import wandb
import tqdm
import numpy as np
import shutil
from diffusion_policy.workspace.base_workspace import BaseWorkspace
from diffusion_policy.policy.robomimic_lowdim_policy import RobomimicLowdimPolicy
from diffusion_policy.dataset.base_dataset import BaseLowdimDataset
from diffusion_policy.env_runner.base_lowdim_runner import BaseLowdimRunner
from diffusion_policy.common.checkpoint_util import TopKCheckpointManager
from diffusion_policy.common.json_logger import JsonLogger
from diffusion_policy.common.pytorch_util import dict_apply, optimizer_to
from torch.utils.data import DataLoader, Sampler 

OmegaConf.register_new_resolver("eval", eval, replace=True)


class CustomIndicesSampler(Sampler):
    def __init__(self, custom_indices):
        self.custom_indices = np.random.permutation(custom_indices)

    def __iter__(self): 
        return iter(self.custom_indices)

    def __len__(self):
        return len( self.custom_indices )

def parse_1_data(data):
    """ 
    data: at time t from dataset.
    #each timestamp can contain multiple uid because of obs_horizon
    """
    if 'demo_no' not in data['obs']:
        raise Exception("Please add demo_no and index_in_demo to the obs first.")
    
    demo_nos = data['obs']['demo_no']
    indices_in_demo = data['obs']['index_in_demo']
    return demo_nos, indices_in_demo



In [9]:
class TrainRobomimicLowdimWorkspace(BaseWorkspace):
    include_keys = ['global_step', 'epoch']

    def __init__(self, cfg: OmegaConf):
        super().__init__(cfg)

        # set seed
        seed = cfg.training.seed
        torch.manual_seed(seed)
        np.random.seed(seed)
        random.seed(seed)

        # configure model
        self.model: RobomimicLowdimPolicy = hydra.utils.instantiate(cfg.policy)

        # configure training state
        self.global_step = 0
        self.epoch = 0

In [10]:
def run(self, save_rollout=False, remove_demos=[], segs_toremove={}):
    cfg = copy.deepcopy(self.cfg)

    # resume training
    if cfg.training.resume:
        lastest_ckpt_path = self.get_checkpoint_path()
        if lastest_ckpt_path.is_file():
            print(f"Resuming from checkpoint {lastest_ckpt_path}")
            self.load_checkpoint(path=lastest_ckpt_path)

    # configure dataset
    dataset: BaseLowdimDataset
    # dataset = hydra.utils.instantiate(cfg.task.dataset)

    cfg_dataset={key: value for key, value in cfg.task.dataset.items() }
    cfg_dataset['remove_demos'] = remove_demos
    # dataset= hydra.utils.instantiate(cfg_dataset)


    self.remove_ids={}                    # all the ids to remove for the key
    self.segs_toremove = segs_toremove    #[start,end]

    for key in segs_toremove.keys():
        segs = segs_toremove[key]
        ids = [] 
        for start, end in segs:
            ids.extend(range(start, end + 1))  # Include the end value
        self.remove_ids[key]=ids



    # assert isinstance(dataset, BaseLowdimDataset)
    # train_dataloader = DataLoader(dataset, **cfg.dataloader)


    if len(self.remove_ids)==0:
        print('---------******--------full traj dataset---------******--------')
        dataset = hydra.utils.instantiate(cfg.task.dataset)
        assert isinstance(dataset, BaseLowdimDataset)
        train_dataloader = DataLoader(dataset, **cfg.dataloader) 
    else:
        print('---------++++++++--------partial traj dataset---------++++++++--------') 
        new_config = {key: value for key, value in cfg.task.dataset.items()} 
        # new_config['obs_keys'] = {key: value for key, value in new_config['obs_keys'].items()}
        # obs_shape_meta_config = {key: value for key, value in new_config['obs_keys'].items()}
        # obs_shape_meta_config['demo_no'] = {'shape': [], 'type': 'low_dim'}
        # obs_shape_meta_config['index_in_demo'] = {'shape': [], 'type': 'low_dim'}
        # new_config['obs_keys'] = obs_shape_meta_config
        # print('-----------------new_config obs_keys: ', new_config['obs_keys'])
        new_config['obs_keys'].extend(['demo_no', 'index_in_demo'])  # add helper keys for demo number and index in demo


        # dataset = hydra.utils.instantiate(cfg.task.dataset)
        dataset = hydra.utils.instantiate(new_config)
        assert isinstance(dataset, BaseLowdimDataset)


        # valid_indices =[]  #in the dataset.
        # print('generating valid indices ...')
        # for index in tqdm( range(len(dataset)) ):
        #     data = dataset.__getitem__(index)
        #     demo_no, indices_in_demo = parse_1_data(data) 

        #     assert torch.all( demo_no[0]==demo_no[1] )                 #obs history from same demo
        #     demo_name=f'demo_{int(demo_no[0])}'
        #     ids = indices_in_demo.numpy().astype(int)
            
        #     should_remove = False
        #     if demo_name in self.remove_ids:
        #         should_remove = bool(set(self.remove_ids[demo_name]) & set(ids))
        #     if should_remove: continue 
        #     valid_indices.append(index)

        # print(f'Valid indices: {len(valid_indices)} / {len(dataset)} ')
        # dataset.lowdim_keys.remove('demo_no')
        # dataset.lowdim_keys.remove('index_in_demo')  # remove the added helper keys (demo_no and index in demo)

        # sampler = CustomIndicesSampler(valid_indices)
        # new_config = {key:value for key,value in cfg.dataloader.items()}
        # new_config['shuffle'] = False
        # new_config['sampler'] = sampler 

        # train_dataloader = DataLoader(dataset, **new_config) 
        train_dataloader = DataLoader(dataset, **cfg.dataloader) 
        # -----------------end of partial traj dataloader -----------------------


    

    normalizer = dataset.get_normalizer()

    # configure validation dataset
    val_dataset = dataset.get_validation_dataset()
    val_dataloader = DataLoader(val_dataset, **cfg.val_dataloader)

    self.model.set_normalizer(normalizer)


In [11]:
ws= TrainRobomimicLowdimWorkspace(cfg)


============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['obs']
using obs modality: rgb with keys: []
using obs modality: depth with keys: []
using obs modality: scan with keys: []


In [12]:
# ws.run(save_rollout=False, remove_demos=[], segs_toremove=segs_toremove)

In [13]:
cfg = copy.deepcopy(ws.cfg)
remove_demos=[]

# configure dataset
dataset: BaseLowdimDataset
# dataset = hydra.utils.instantiate(cfg.task.dataset)

cfg_dataset={key: value for key, value in cfg.task.dataset.items() }
cfg_dataset['remove_demos'] = remove_demos

In [14]:
self=ws

self.remove_ids={}                    # all the ids to remove for the key
self.segs_toremove = segs_toremove    #[start,end]

for key in segs_toremove.keys():
    segs = segs_toremove[key]
    ids = [] 
    for start, end in segs:
        ids.extend(range(start, end + 1))  # Include the end value
    self.remove_ids[key]=ids

In [15]:
len(self.remove_ids)

87

In [16]:
new_config = {key: value for key, value in cfg.task.dataset.items()} 
new_config['obs_keys'].extend(['demo_no', 'index_in_demo'])  # add helper keys for demo number and index in demo

In [17]:
# dataset = hydra.utils.instantiate(new_config)
del new_config['_target_']

from diffusion_policy.dataset.robomimic_replay_lowdim_dataset import RobomimicReplayLowdimDataset
dataset = RobomimicReplayLowdimDataset(**new_config)

------------total 300 demos, hdf5_filter_key=None------------


Loading hdf5 to ReplayBuffer: 100%|██████████| 300/300 [00:00<00:00, 900.11it/s]


In [18]:
dataset

In [19]:
print('---------++++++++--------partial traj dataset---------++++++++--------') 

# new_config['obs_keys'] = {key: value for key, value in new_config['obs_keys'].items()}
# obs_shape_meta_config = {key: value for key, value in new_config['obs_keys'].items()}
# obs_shape_meta_config['demo_no'] = {'shape': [], 'type': 'low_dim'}
# obs_shape_meta_config['index_in_demo'] = {'shape': [], 'type': 'low_dim'}
# new_config['obs_keys'] = obs_shape_meta_config
# print('-----------------new_config obs_keys: ', new_config['obs_keys'])


# dataset = hydra.utils.instantiate(cfg.task.dataset)

assert isinstance(dataset, BaseLowdimDataset)

---------++++++++--------partial traj dataset---------++++++++--------
